<a href="https://colab.research.google.com/github/jedias/BI-Master/blob/main/TrabalhoFinal_BI_Master_Documentos_T%C3%A9cnicos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Introdução
O procesasmento de linguagem natural através dos metodos de IA é um problema que tem recebido bastante atenção nos ultimos anos. Sua aplicação abrange desde a construção de chatterbots até tradução de texto. 
Esse trabalho busca demonstrar a viabilidade da geração automática de documentos a partir de tabelas factuais. 



In [ ]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

#Aquisição dos dados

Inicalmente é necessária a escolha de um corpus de referência. Nesse caso o corpus escolhido será o conjunto de normas técnicas disponível em https://canalfornecedor.petrobras.com.br/pt/regras-de-contratacao/catalogo-de-padronizacao/#especificacoes-tecnicas


Parsing os documentos PDF

In [ ]:
!pip install PyPDF2
 
#importando as bilibliotecas usadas no parsing

import bz2
import zipfile
import PyPDF2
from glob2 import glob

In [ ]:
def pdf2pandas(local):
    docn = 0
    df = pd.DataFrame({'item':[] ,'text':[]})
    filelist = glob(local+'/**/*.PDF',recursive=True)
    
    for item in filelist:
      
      with open(item,'rb') as pdf_file:
        read_pdf = PyPDF2.PdfFileReader(item)
        number_of_pages = read_pdf.getNumPages()
        docn = docn + 1
        docx = ''
        
        for page_number in range(number_of_pages):  
          page = read_pdf.getPage(page_number)
          page_content = page.extractText()
          docx = docx + page_content 
            
        df.loc[docn,'text'] = docx
        df.loc[docn,'item'] = item
        print(df.tail(1))
    return df

In [ ]:
npt = pdf2pandas('/content/drive/My Drive/Colab Notebooks/TrabalhoBi/normas/')
npt.to_csv('/content/drive/My Drive/Colab Notebooks/TrabalhoBi/normas/ptbr.csv') 
nin = pdf2pandas('/content/drive/My Drive/Colab Notebooks/TrabalhoBi/NI/')
nin.to_csv('/content/drive/My Drive/Colab Notebooks/TrabalhoBi/NI/english.csv')

Após o parsing dos documentos, eles foram salvos em CSV. Essa passo economiza tempo na retomada desses textos no futuro. 

In [ ]:
npt = pd.read_csv('/content/drive/My Drive/Colab Notebooks/TrabalhoBi/normas/ptbr.csv')
npt.drop('Unnamed: 0', axis='columns', inplace=True)
nin = pd.read_csv('/content/drive/My Drive/Colab Notebooks/TrabalhoBi/NI/english.csv')
nin.drop('Unnamed: 0', axis='columns', inplace=True)

In [ ]:
npt.item.replace({'/content/drive/My Drive/Colab Notebooks/TrabalhoBi/normas/':''}, regex=True, inplace = True)
nin.item.replace({'/content/drive/My Drive/Colab Notebooks/TrabalhoBi/NI/':''}, regex=True, inplace = True)
nin.item.replace({'I':''}, regex=True, inplace = True)
corpus = npt.merge(nin, how='outer', left_on='item', right_on='item')
corpus.dropna(inplace = True)
corpus['text_x'] = corpus['text_x'].str.lower()
corpus['text_y'] = corpus['text_y'].str.lower()
corpus.to_excel('/content/drive/My Drive/Colab Notebooks/TrabalhoBi/corpus.xlsx')
corpus.head()

Continuando...

In [ ]:
corpus = pd.read_excel('/content/drive/My Drive/Colab Notebooks/TrabalhoBi/corpus.xlsx')
corpus.drop('Unnamed: 0', axis='columns', inplace=True)
corpus.replace({'\n':' '}, regex=True, inplace = True)

In [ ]:
corpus.head()

# Modelamento de linguagem

# 1. Pre-processamento

# Tokenização do texto




In [ ]:
import numpy as np
import nltk
nltk.download('punkt')

In [ ]:
#mantendo documentos individuais na sua coluna
corpus['Xsent'] = corpus.text_x.apply(lambda row: nltk.sent_tokenize(row))
corpus['Ysent'] = corpus.text_y.apply(lambda row: nltk.sent_tokenize(row))
corpus['Xtk'] = corpus.text_x.apply(lambda row: nltk.word_tokenize(row, language = 'portuguese'))
corpus['Ytk'] = corpus.text_y.apply(lambda row: nltk.word_tokenize(row, language = 'english'))

In [ ]:
corpus.head()

#Treinamento dos Word2Vec


In [ ]:
from gensim.models import Word2Vec

In [ ]:
modelX = Word2Vec(sentences = corpus.Xtk, size = 300, window = 50, min_count=1, workers=100)
modelY  = Word2Vec(sentences = corpus.Ytk, size = 300, window = 50, min_count = 1, workers = 100)

In [ ]:
modelX.save("/content/drive/My Drive/word2vec.pt.model")
modelY.save("/content/drive/My Drive/word2vec.en.model")

In [ ]:
modelX = Word2Vec.load("/content/drive/My Drive/word2vec.pt.model")
modelY =  Word2Vec.load("/content/drive/My Drive/word2vec.en.model")

# Conhecendo os dados

[Fonte](https://towardsdatascience.com/multi-class-text-classification-with-doc2vec-logistic-regression-9da9947b43f4)

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
from gensim.models import Doc2Vec
from sklearn import utils
from sklearn.model_selection import train_test_split
import gensim
from sklearn.linear_model import LogisticRegression
from gensim.models.doc2vec import TaggedDocument
import re
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
def display_pca_scatterplot(model, words=None, sample=0):
    if words == None:
        if sample > 0:
            words = np.random.choice(list(model.wv.vocab.keys()), sample)
        else:
            words = [ word for word in model.wv.vocab ]
        
    word_vectors = np.array([model.wv[w] for w in words])

    twodim = PCA().fit_transform(word_vectors)[:,:2]
    
    plt.figure(figsize=(24,16))
    plt.scatter(twodim[:,0], twodim[:,1], edgecolors='k', c='r')
    for word, (x,y) in zip(words, twodim):
        plt.text(x+0.05, y+0.05, word)

In [ ]:
display_pca_scatterplot(model = modelX, words = modelX.wv.vocab.keys(),sample = 50)

In [ ]:
display_pca_scatterplot(modelX,['bomba','compressor','vaso','manutenção','operação','solda','selo','sobressalente','confiabilidade'] )
display_pca_scatterplot(modelY,['pump','compressor','vessel','maintenance','operation','weld','seal','extra','spare','reliability'])

In [ ]:
display_pca_scatterplot(modelX,['n-1959','n-2201','n-2508','n-0898','bomba','compressor'])
display_pca_scatterplot(modelX,['n-1959','n-2201','n-2508','n-0898','pump', 'compressor'])

In [ ]:
modelX.wv.similar_by_word('petrobras')

In [ ]:
modelX.wv.similar_by_vector(modelX.wv['petrobras']-modelX.wv['produção'])

In [ ]:
modelX.wv.similar_by_vector(modelX.wv['petrobras']-modelX.wv['óleo']-modelX.wv['gás']-modelX['produção'])

In [ ]:
modelX.wv.similar_by_vector(modelX.wv['confiabilidade']+modelX.wv['manutenção'])

In [ ]:
modelX.wv.similar_by_vector(modelX.wv['confiabilidade']-modelX.wv['sobressalente'])

In [ ]:
modelX.most_similar(positive = ['bomba','confiabilidade'],negative = 'spare')

# Tradução W2V

In [ ]:
from gensim.models.translation_matrix import TranslationMatrix

In [ ]:
model_trans = TranslationMatrix(modelX.wv, modelY.wv)

In [ ]:
#lista de palavras
word_pairs = [('petrobras','petrobras'), ('inspeção','inspection'),
              ('elétrica','electrical'),('sistema','system'), 
              ('responsável','responsible'),
              ('motor','motor'),('elétrico','electric'),
              ('solda','welding'),
              ('cabo','cable'), ('fio','wire'),
              ('tubo','pipe'),
              ('superfície','surface'),('norte','north'),
              ('número','number'), ('dimensão','dimension'),
              ('mínimos','minimum'),
              ('mm','mm'), ('m³','m³'),
              ('secundário', 'secondary'),
              ('compressor','compressor'),('compressores','compressors'),
              ('são','are'), ('é','is'), ('podem','may'),
              ('petróleo','oil'), ('combustível','fuel'),
              ('o','the'), ('mesmas','same'),
              ('do','of'),
              ('maior','larger'),('detalhes','details'),
              ('para','for'),('com','with'),
              ('(',"("), (',',","),('caldeira','boiler'),
              ('nota','note'),('todos','all'),('todas','all'),
              ('mesmo','same'), ('verde','green'),('azul','blue'),
              ('água','water'),('óleo','oil'), ('substitui','replaces'),
              ('apresentados','presented'), ('tendo','having'),
              ('temporários','temporary'), ('desenhos','drawings'), 
              ('seqüências','sequences'),('seqüência','sequence'),
              ('previstas','foreseen'), ('volume','volume'),
              ("/",'/'), 
              ('n-2201','n-2201'),
              ('propostas','proposals'), ('tabelas','tables'),
              ('um', 'a'), ('uma','a'), ('esta','this'),('norma','standard'),
              ('contém','contains'), 
              ('requisitos','requirements'), ("e",'and'), 
              ('recomendadas','recommended'), ('práticas','practices'),
              ('padrão','standard'),('norma','normative'),('incluem',"include"),
              ('anexos','annexes'),('nota','note'),
              ('localização','location'),
              ('autora','author'),('similares','similar'),
              ('existe','is'),
              ('detalhamento','detailing'),
              ('projetos','projects'),
              ('página','page'),('páginas','pages'),
              ('apresentação','presentation'), ('carga','cargo'),
              ('fases',"phases"),('cimento','cement'),('água/cimento','water/cement'),
              ('acabamento','finishing'),('superfícies','surfaces'),
              ('4.1.5','4.1.5'), ('8.2.2.2','8.2.2.2'),('4.6.2.3','4.6.2.3'),
              ('4.6.2.3','4.6.2.3'), 
              ('estruturas','structures'),('estrutural','structural'),
              ('ressonância','resonance'), ('defensa','shield'),
              ('concreto','concrete'),('estão','are'),('aço','steel'),
              ('pelo','by')]

In [ ]:
model_trans.train(word_pairs=word_pairs)
model_trans.translate(['tubulação','bomba'],topn=5)

In [ ]:
model_trans.translate(corpus.Xtk[0][50:150],topn=3)

In [ ]:
#infelizmente os resultados são ruins


# Buscas **W2V**

In [ ]:
modelX.most_similar(positive = ['manutenção','bomba'])

In [ ]:
modelY.most_similar(positive = ['pump','maintenance'])

Mesmo partindo da tradução manual dos documentos, os contextos em cada lígua não idênticos.
Isso pode indicativo de desintendimentos.

# Modelo GloVe

In [ ]:
#https://radimrehurek.com/gensim/scripts/glove2word2vec.html

from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec

In [ ]:
import gensim.downloader

In [ ]:
#testando GloVe 300d sem treino
# Download the "glove" embeddings
glove_vectors = gensim.downloader.load('glove-wiki-gigaword-300')

In [ ]:
# Use the downloaded vectors as usual:
glove_vectors.most_similar('petrobras')

In [ ]:
glove_vectors.most_similar(positive ='maintenance')

In [ ]:
glove_vectors.most_similar(positive =['maintenance', 'petrobras'])

In [ ]:
glove_vectors.most_similar(positive = ['pump','reliability'],negative = ['spare'])

In [ ]:
#pedidos da Zelda
glove_vectors.most_similar(positive = ['cute','cat'])

In [ ]:
glove_vectors.most_similar(positive = ['reptile','venomous'],negative = ['limbs'])

In [ ]:
glove_vectors.most_similar(positive = ['gray','mammal','africa','asia','trunk','animal'])

In [ ]:
glove_vectors.train(corpus.tkY)

Infelizemente o modelo GloVe pré treinado não a continuaçao do treinamento com nossos documentos.

Mas isso pode ser contornado:

In [ ]:
#Modelo GloVe + Normas
glove_file = '/content/drive/My Drive/glove.6B.50d.txt'
tmp_file = '/content/drive/My Drive/Colab Notebooks/TrabalhoBi/normas/english.csv'
_ = glove2word2vec(glove_file, tmp_file)

gmodel = KeyedVectors.load_word2vec_format(tmp_file)
#gmodel.save('/content/drive/My Drive/word2glovec.en.model')

In [ ]:
gmodel= KeyedVectors.load('/content/drive/My Drive/word2glovec.en.model')

In [ ]:
gmodel.most_similar('petrobras')

In [ ]:
gmodel.wv.similar_by_vector(gmodel.wv['pizza']-gmodel.wv['filling'])

In [ ]:
gmodel.wv.words_closer_than('reliability','maintainability')

In [ ]:
gmodel.similar_by_word('reliability')

In [ ]:
gmodel.words_closer_than('reliability','maintainability')

# Tradução GloVe

In [ ]:
from gensim.models.translation_matrix import TranslationMatrix

In [ ]:
#foram excluídas as palavras não reconhecidas pelo GloVe

word_pairs = [('petrobras','petrobras'), ('propriedade','property'),
              ('substitui','replaces'),('cancela','cancels'),
              ('inspeção','inspection'),
              ('elétricas','electrical'),('sistema','system'), 
              ('motor','motor'),('elétrico','electric'),
              ('solda','welding'), ('cabo','cable'),
              ('expostas','exposed'),
              ('bomba','pump'), ('tubo','pipe'),
              ('superfície','surface'),('norte','north'),
              ('número','number'), ('dimensão','dimension'),
              ('mínimos','minimum'),
              ('mm','mm'), 
              ('secundário', 'secondary'),
              ('compressor','compressor'),
              ('são','are'), ('é','is'), ('podem','may'),
              ('petróleo','oil'), ('combustível','fuel'),
              ('o','the'), ('mesmas','same'),
              ('de','of'),('do','of'),('da','of'),
              ('maior','larger'),('detalhes','details'),
              ('para','for'),('com','with'),
              ('caldeira','boiler'),
              ('nota','note'),('todos','all'),('todas','all'),
              ('mesmo','same'), ('verde','green'),('azul','blue'),
              ('água','water'),('óleo','oil'), ('substitui','replaces'),
              ('apresentados','presented'), ('tendo','having'),
              ('temporários','temporary'),
              ('desenhos','drawings'), ('seqüências','sequences'),
              ('previstas','foreseen'), 
              ("/",'/'), ('(',"("), (',',","),(")",")"),('.','.'),
              ('propostas','proposals'), ('tabelas','tables'),
              ('um', 'a'), ('uma','a'), ('esta','this'),('norma','standard'),
              ('contém','contains'), ('técnicos','technical'),
              ('requisitos','requirements'), ("e",'and'), 
              ('recomendadas','recommended'), ('práticas','practices'),
              ('padrão','standard'),('norma','normative'),('incluem',"include"),
              ('anexos','annexes'),('nota','note'),
              ('localização','location'),
              ('autora','author'),('similares','similar'),
              ('içamentos','lifting'), ('existe','is'),
              ('travejamentos','supporting'), ('detalhamento','detailing'),
              ('fases',"phases"),('cimento','cement'),
              ('acabamento','finishing'),('superfícies','surfaces'),
              ('ressonância','resonance'),('por','by'),('projetos','projects'),
              ('página','page'),('páginas','pages'),('apresentação','presentation'),
              ('carga','cargo'),('1','1'),('concreto','concrete'),
              ('anterior','previous'),('à','to'),('desta','this')]

In [ ]:
model_trans_glove = TranslationMatrix(modelX.wv,gmodel)
model_trans_glove.train(word_pairs=word_pairs)
model_trans_glove.save("/content/drive/My Drive/glove.doc2vec.x.model")

In [ ]:
model_trans_glove = TranslationMatrix.load("/content/drive/My Drive/glove.doc2vec.x.model")

In [ ]:
#tradução palavra a palavra
for doc in range(0):
  for word in range(len(corpus.Xtk[doc])):
      try:  print(model_trans_glove.translate(corpus.Xtk[doc],topn=3))
      except:  print(word)

In [ ]:
model_trans_glove.translate(corpus.Xtk[0][50:150],topn=3)

... um pouco melhor mas ainda insuficiente. Poderíamos continuar acrescentando sinônimos mas isso consome tempo. 

## Geraçao de glossário: Ingles técnico - GloVe

In [ ]:
#gerando lista de palavras conhecidas
word_pairs1 = []
for line in range(len(word_pairs)):
  word_pairs1 = word_pairs1 + [(word_pairs[line][1],word_pairs[line][1])]

In [ ]:
word_pairs1

In [ ]:
model_trans_glove = TranslationMatrix(modelY.wv,glove_vectors)
model_trans_glove.train(word_pairs=word_pairs1)

In [ ]:
model_trans_glove.translate(["pipe",'pump','oil','pressure'],topn=3)

In [ ]:
for doc in range(len(corpus)):
    for word in corpus.Ytk[doc]:
      try: print(model_trans_glove.translate(word,topn=3))
      except: print(word)

Por esse método conseguimos obter alguns sinônimos mas ainda com desempenho insatisfatório.

## Gensim - Doc2Vec

In [ ]:
#https://medium.com/@mishra.thedeepak/doc2vec-simple-implementation-example-df2afbbfbad5
#Import all the dependencies
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize

In [ ]:
data = corpus.item + corpus.text_x

tagged_data = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(data)]

max_epochs = 100
vec_size = 300
alpha = 0.025

modelo = Doc2Vec(vector_size=vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                dm =1)
  
modelo.build_vocab(tagged_data)

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    modelo.train(tagged_data,
                total_examples=modelo.corpus_count,
                epochs=modelo.epochs)
    # decrease the learning rate
    modelo.alpha -= 0.0002
    # fix the learning rate, no decay
    modelo.min_alpha = modelo.alpha

modelo.save("/content/drive/My Drive/pt.d2v.model")
print("Model Saved")

In [ ]:
data = corpus.item + corpus.text_y
tagged_data = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(data)]

max_epochs = 100
vec_size = 300
alpha = 0.025

model = Doc2Vec(vector_size=vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                dm =1)
  
model.build_vocab(tagged_data)

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=model.epochs)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha

model.save("/content/drive/My Drive/en.d2v.model")
print("Model Saved")

In [ ]:
#busca de sentenças
new_sentence = 'confiabilidade de equipamentos'
modelo.docvecs.most_similar([modelo.infer_vector(new_sentence)])

In [ ]:
corpus.text_x[118][100:150]

## Tradução Doc2Vec

In [ ]:
# Continuando
from gensim.models.doc2vec import Doc2Vec

modelo= Doc2Vec.load("/content/drive/My Drive/pt.d2v.model")
model= Doc2Vec.load("/content/drive/My Drive/en.d2v.model")

In [ ]:
from gensim.models.translation_matrix import TranslationMatrix
model_trans = TranslationMatrix(modelo.wv, model.wv)
model_trans.train(word_pairs=word_pairs)

In [ ]:
model_trans.save("/content/drive/My Drive/doc2vec.x.model")

In [ ]:
model_trans.translate(['temperatura','petrobras','norma','tubulação','água'],topn=3)

Notamos um desepenho aparentemente pior para essa aplicação quando comparado ao modelo W2V

In [ ]:
modelo.wv.similar_by_word('bomba')

In [ ]:
modelo.wv.similarity('bomba','voluta')

#Treinamento da Tradução

In [ ]:
!pip install keras_tqdm

In [ ]:
from keras.layers.core import Activation, Dense, Dropout, SpatialDropout1D
from keras.layers import Input
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM , GRU, SimpleRNN
from keras.models import Sequential, Model
from keras.preprocessing import sequence
from sklearn.model_selection import train_test_split
import collections
import matplotlib.pyplot as plt
import nltk
import numpy as np
import os
from keras_tqdm import TQDMNotebookCallback
from keras.callbacks import ModelCheckpoint
from keras.layers.wrappers import TimeDistributed
import nltk
nltk.download('punkt')

In [ ]:
import tensorflow as tf
import torch

In [ ]:
train = corpus[:10]
train['Xvec'] = train.Ytk.apply(lambda row: modelY.wv[row])
#train['gvec'] = train.tken.apply(lambda row: gmodel.wv[row])  #falha por palavras desconhecidas
train['Yvec'] = train.Xtk.apply(lambda row: modelX.wv[row])
train.head()

##LSTM 
input considerando todo o documento como um arranjo de vetores

In [ ]:
from __future__ import print_function
from keras import regularizers, optimizers 
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Dropout, Embedding, Flatten, Bidirectional, Input, LSTM, Reshape
from keras.callbacks import EarlyStopping,ModelCheckpoint
from keras.optimizers import Adam
from keras.metrics import categorical_accuracy, mean_squared_error, mean_absolute_error
from keras.layers.normalization import BatchNormalization
import tensorflow as tf

In [ ]:
X = np.array(train.Xvec)
Y = np.array(train.Yvec)

In [ ]:
from tensorflow.keras import preprocessing

In [ ]:
paddedX = sequence.pad_sequences(X)
paddedY = sequence.pad_sequences(Y)
X = np.expand_dims(paddedX, axis = np.zeros(300))
print('Input Shape: ', X.shape) 
Y = np.expand_dims(paddedY, axis = np.zeros(300))
print('Output Shape', Y.shape)

In [ ]:
from keras.models import Sequential  
from keras.layers import *  

IAmodel = Sequential(
    [
        Dense(2, activation="relu", name="layer1"),
        Dense(3, activation="relu", name="layer2"),
        Dense(4, name="layer3"),
    ]
)

IAmodel.add(Reshape(Y.shape))
IAmodel.compile(loss="binary_crossentropy", optimizer=optimizers.SGD(lr=0.5))

In [ ]:
tradutor = IAmodel.fit(X, Y, batch_size = 1, epochs=100)

As dimensões de entrada e saída variam do uma língua para outra, inviabilizando o uso do modelo sequencial


## Enconder/Decoder LSTM
Material de aula +
https://blog.keras.io/a-ten-minute-introduction-to-sequence-to-sequence-learning-in-keras.html

In [ ]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
import nltk
nltk.download('punkt')

In [ ]:
corpus = pd.read_excel('/content/drive/My Drive/Colab Notebooks/TrabalhoBi/corpus.xlsx')
corpus.drop('Unnamed: 0', axis='columns', inplace=True)

In [ ]:
corpus = corpus.replace(to_replace= r'*\n*',value='') #nao funcionou nao sei pq

In [ ]:
#mantendo documentos individuais na sua coluna
X = corpus.text_x.apply(lambda row: nltk.sent_tokenize(row,language = 'portuguese'))
Y = corpus.text_y.apply(lambda row: nltk.sent_tokenize(row,language = 'english'))

In [ ]:
X = pd.DataFrame(X.to_list())
Y = pd.DataFrame(Y.to_list())

In [ ]:
def AddTags(sentence):
  return 'sent/  ' + sentence+ ' /sent'

In [ ]:
X = X.apply(lambda sent: AddTags(sent))
Y = Y.apply(lambda sent: AddTags(sent))

In [ ]:
X.fillna(" ",inplace=True)
Y.fillna(" ",inplace=True)

In [ ]:
A = X.apply(lambda row: row.str.cat(sep=' '),axis = 1)

B = Y.apply(lambda row: row.str.cat(sep=' '),axis = 1)

In [ ]:
A.replace('\n',' ')

In [ ]:
X=A
Y=B

In [ ]:
for row in range(len(X)):
  X[row]= nltk.word_tokenize(X[row], language = 'portuguese')
for row in range(len(Y)): 
  Y[row] = nltk.word_tokenize(Y[row], language = 'english')

In [ ]:
X.head()

In [ ]:
from gensim.models import Word2Vec

In [ ]:
mdlX = Word2Vec(sentences = X, size = 300, window = 100, min_count=1, workers=100)
mdlY  = Word2Vec(sentences = Y, size = 300, window = 100, min_count = 1, workers = 100)

In [ ]:
mdlX.wv.similar_by_word('n-2201')

In [ ]:
mdlY.wv.similar_by_word('n-2201')

In [ ]:
Xemb = mdlX.wv.get_keras_embedding(train_embeddings=False)
Yemb = mdlY.wv.get_keras_embedding(train_embeddings=False)

In [ ]:
array_of_word_lists= X

In [ ]:
array_of_word_lists_AFTER_being_transformed_by_word2vec = X.apply(lambda word: mdlX.wv[word])

In [ ]:
source_word_indices = []
for i in range(len(array_of_word_lists)):
    source_word_indices.append([])
    for j in range(len(array_of_word_lists[i])):
        word = array_of_word_lists[i][j]
        if word in mdlX.wv.vocab:
            word_index = mdlX.wv.vocab[word].index
            source_word_indices[i].append(word_index)
        else:
            # Do something. For example, leave it blank or replace with padding character's index.
            source_word_indices[i].append(padding_index)
source = np.array(source_word_indices)

In [ ]:
from keras.models import Sequential
from keras.layers import Embedding, GRU, Input, Flatten, Dense, TimeDistributed, Activation, PReLU, RepeatVector, Bidirectional, Dropout
from keras.optimizers import Adam, Adadelta
from keras.callbacks import ModelCheckpoint
from keras.losses import sparse_categorical_crossentropy, mean_squared_error

keras_model = Sequential()
keras_model.add(Xemb)
keras_model.add(Bidirectional(GRU(300, return_sequences=True, dropout=0.1, recurrent_dropout=0.1, activation='tanh')))
keras_model.add(TimeDistributed(Dense(600, activation='tanh')))
# keras_model.add(PReLU())
# ^ For some reason I get error when I add Activation ‘outside’:
# int() argument must be a string, a bytes-like object or a number, not 'NoneType'
# But keras_model.add(Activation('relu')) works.
#keras_model.add(Dense(source_arr.shape[1] * source_arr.shape[2]))
# size = max-output-sentence-length * embedding-dimensions to learn the embedding vector and find the nearest word in word2vec_model.wv.similar_by_vector() afterwards.
# Alternatively one can use Dense(vocab_size) and train the network to output one-hot categorical words instead.
# Remember to change Keras loss to sparse_categorical_crossentropy.
# But this won’t benefit from Word2Vec. 

keras_model.compile(loss=mean_squared_error,
              optimizer=Adadelta(),
              metrics=['mean_absolute_error'])
keras_model.summary()

In [ ]:
filepath="best-weights.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_mean_absolute_error', verbose=1, save_best_only=True, mode='auto')
callbacks_list = [checkpoint]
keras_model.fit(array_of_word_lists, array_of_word_lists_AFTER_being_transformed_by_word2vec, epochs=100, batch_size=2000, shuffle=True, callbacks=callbacks_list, validation_split=0.2)

In [ ]:
filepath="best-weights.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_mean_absolute_error', verbose=1, save_best_only=True, mode='auto')
callbacks_list = [checkpoint]
keras_model.fit(array_of_word_lists, array_of_word_lists_AFTER_being_transformed_by_word2vec, epochs=100, batch_size=2000, shuffle=True, callbacks=callbacks_list, validation_split=0.2)

In [ ]:
rows, columns = X.shape
for row in range(rows):
  for column in range(columns):
    X.iloc[row][0] = X.iloc[row][0] + X.iloc[row][0]

In [ ]:
X.head()

In [ ]:
rows, columns = X.shape
for row in range(rows):
  for column in range(columns):
    X[row][0] = X[row][0] + X[row][0]

In [ ]:
from gensim.models import Word2Vec

In [ ]:
for column in X.columns:
  modelX = Word2Vec.train(sentences = X[columns], size = 300, window = 10, min_count=2, workers=100)

In [ ]:
for column in Y.columns:
  modelY = Word2Vec.train(sentences = Y[columns])

In [ ]:
modelX.wv.similar_by_vector(np.zeros(300))

In [ ]:
modelY.wv.similar_by_vector(np.zeros(300))

In [ ]:
len(modelX.wv.vocab.keys())

In [ ]:
#Padding
for col in X.columns:
  for row in range(len(X)):
      try:
        X[col][row] =  modelX.wv[X[col][row]]
      except: 
        [np.zeros(300)]

In [ ]:
for col in Y.columns:
  for row in range(len(Y)):
      try:
        Y[col][row] =  modelY.wv[Y[col][row]]
      except: 
        [np.zeros(300)]

In [ ]:
X.tail()

In [ ]:
ndocs, num_encoder_tokens = X.shape
ndocs, num_decoder_tokens = Y.shape

In [ ]:
num_encoder_tokens , num_decoder_tokens

In [ ]:
latent_dim = 128
encoder_input_data = pd.DataFrame(X)
decoder_input_data = encoder_input_data
decoder_target_data = pd.DataFrame(Y)

In [ ]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens)
#encoder = LSTM(latent_dim, return_state=True)
#encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
#encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the 
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [ ]:
# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], validation_data=decoder_input_data, batch_size=1, epochs=5)

In [ ]:
modelY.wv.vocab[np.zeros(300)]

In [ ]:
fr = lines.fr.apply(lambda x : 'START_ '+ x + ' _END')
# Create vocabulary of words
all_eng_words=set()
for eng in lines.eng:
    for word in eng.split():
        if word not in all_eng_words:
            all_eng_words.add(word)
    
all_french_words=set()
for fr in lines.fr:
    for word in fr.split():
        if word not in all_french_words:
            all_french_words.add(word)
input_words = sorted(list(all_eng_words))
target_words = sorted(list(all_french_words))
num_encoder_tokens = len(all_eng_words)
num_decoder_tokens = len(all_french_words)
# del all_eng_words, all_french_words

https://stackoverflow.com/questions/51492778/how-to-properly-use-get-keras-embedding-in-gensim-s-word2vec

In [ ]:
from gensim.models import Phrases